In [308]:
from glob import glob
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from nltk.corpus import stopwords
import nltk
import os
import glob
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

In [309]:
nltk.download("stopwords")
STOPWORD_SET = set(
    stopwords.words("english")
)  # set of stopwords of the English language
PUNCTUATION_SET = set("!\"#$%&'()*+,-./:;<=>?@[\\]^_`{|}~")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\39392\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [310]:
nlp = spacy.load('en_core_web_sm')
document = 'MachineLearning.txt'

In [311]:
for filepath in glob.glob(document):
    with open(filepath,'r', encoding = 'utf-8', errors='ignore') as f:
        text = f.read()
        lines = text.split(' , ')
        for line in lines:
            line = nlp(line)
            #print(nlp(text))
            
            tokens = [token.text for token in line]
            #print (tokens)
       

In [312]:
#conto le parole piu frequenti all'interno del mio txt. La frequenza delle parole mi sara utile per fare la summarization finale
word_frequency = {}

for word in line: 
    if word.text.lower() not in STOPWORD_SET:
        if word.text.lower() not in PUNCTUATION_SET:
            if word.text not in word_frequency.keys():
                word_frequency[word.text] = 1
            else:
                word_frequency[word.text] = word_frequency[word.text]+1

In [313]:
#word_frequency

In [314]:
#vado a vedere qual e' la parola che appare piu frequentemente. Questo poi lo vado a dividere per ogni parola in modo da avere un punteggi a disposizione per poi la summarization
max_frequency = max(word_frequency.values())
#print(max_frequency)

for word in word_frequency.keys():
    word_frequency[word]= (word_frequency[word]/max_frequency)

#print(word_frequency)

In [315]:
#faccio la sentence tokenization
sentence = [sent for sent in line.sents]
#print(sentence)

#creo dei punteggi per ogni frase. SUlla base dei punteggi posso definire la summarization
sentence_score = {}
score = {}
for sent in sentence:
    for word in sent:
        if word.text.lower() in word_frequency.keys():
            if sent not in sentence_score.keys():
                sentence_score[sent] = word_frequency[word.text.lower()]
            else:
                sentence_score[sent] += word_frequency[word.text.lower()]             
print("\nSentences :", sentence_score)


Sentences : {Machine learning (ML) is a field of inquiry devoted to understanding and building methods that 'learn', that is, methods that leverage data to improve performance on some set of tasks.: 4.0555555555555545, It is seen as a part of artificial intelligence.: 0.2222222222222222, Machine learning algorithms build a model based on sample data, known as training data, in order to make predictions or decisions without being explicitly programmed to do so.: 4.277777777777777, Machine learning algorithms are used in a wide variety of applications, such as in medicine, email filtering, speech recognition, and computer vision, where it is difficult or unfeasible to develop conventional algorithms to perform the needed tasks.: 2.944444444444442, A subset of machine learning is closely related to computational statistics, which focuses on making predictions using computers, but not all machine learning is statistical learning.: 4.8888888888888875, The study of mathematical optimization

In [316]:
from heapq import nlargest

#faccio una compressione del testo del 40%
select_lenght = int(len(sentence)*0.4)

summary = nlargest(select_lenght, sentence_score, key=sentence_score.get)

In [317]:
summarized = [word.text for word in summary]

summary = ' '.join(summarized)

print(summary)

Machine learning and data mining often employ the same methods and overlap significantly, but while machine learning focuses on prediction, based on known properties learned from the training data, data mining focuses on the discovery of (previously) unknown properties in the data (this is the analysis step of knowledge discovery in databases). Data mining uses many machine learning methods, but with different goals; on the other hand, machine learning also employs data mining methods as "unsupervised learning" or as a preprocessing step to improve learner accuracy. Much of the confusion between these two research communities (which do often have separate conferences and separate journals, ECML PKDD being a major exception) comes from the basic assumptions they work with: in machine learning, performance is usually evaluated with respect to the ability to reproduce known knowledge, while in knowledge discovery and data mining (KDD) the key task is the discovery of previously unknown kn

In [318]:
#computing blue rouge score:
rouge = Rouge()
#print("Blue Score : ", sentence_bleu([document], summary, weights=(1,0,0,0)))

rouge_score = rouge.get_scores(' '.join(summary), ' '.join(document))
print("Rouge Score :", rouge_score)

Rouge Score : [{'rouge-1': {'r': 1.0, 'p': 0.3, 'f': 0.4615384579881657}, 'rouge-2': {'r': 0.6875, 'p': 0.034482758620689655, 'f': 0.06567164088144353}, 'rouge-l': {'r': 1.0, 'p': 0.3, 'f': 0.4615384579881657}}]
